<a href="https://colab.research.google.com/github/versant2612/jnotebooks/blob/main/kgtk/03_kg_graph_embeddingsRESCAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kgtk==1.0.1

     |████████████████████████████████| 550 kB 9.6 MB/s 
     |████████████████████████████████| 407 kB 56.5 MB/s 
     |████████████████████████████████| 203 kB 54.4 MB/s 
     |████████████████████████████████| 78 kB 5.7 MB/s 
     |████████████████████████████████| 99 kB 7.7 MB/s 
     |████████████████████████████████| 119 kB 67.2 MB/s 
     |████████████████████████████████| 10.1 MB 55.7 MB/s 
     |████████████████████████████████| 57 kB 3.8 MB/s 
     |████████████████████████████████| 88 kB 7.2 MB/s 
     |████████████████████████████████| 40 kB 4.5 MB/s 
     |████████████████████████████████| 130 kB 54.6 MB/s 
     |████████████████████████████████| 21.5 MB 6.0 MB/s 
     |████████████████████████████████| 81 kB 7.6 MB/s 
     |████████████████████████████████| 1.2 MB 33.1 MB/s 
     |████████████████████████████████| 167 kB 69.5 MB/s 
     |████████████████████████████████| 1.8 MB 46.2 MB/s 
     |████████████████████████████████| 78 kB 5.8 MB/s 
     |██████████████████████

# KGTK graph-embeddings

`kgtk graph-embeddings` command takes as input a KGTK edge file and compute the embeddings of the graph nodes and relations.

Please refer to [graph-embeddings documentation](https://kgtk.readthedocs.io/en/latest/analysis/graph_embeddings/) for further details.

In [1]:
import os
from kgtk.configure_kgtk_notebooks import ConfigureKGTK
from kgtk.functions import kgtk, kypher
from gensim.models import KeyedVectors
import tempfile
import pandas as pd
import numpy as np
import h5py, torch
from torchbiggraph.model import ComplexDiagonalDynamicOperator, DotComparator, CosComparator
import json

In [2]:
# Parameters

# Folder on local machine where to create the output and temporary folders
input_path = None
output_path = "/tmp/projects"
project_name = "tutorial-graph-embeddings"
input_files_url="https://github.com/usc-isi-i2/kgtk-tutorial-files/raw/main/datasets/arnold-profiled"

#### Define a custom json files config to download the file `derived_P31x` as it is not part of default download list

In [3]:
extra_files_config = {
    "derived_P31x": "derived.P31x.tsv"
    }

open('/root/extra_files.json', 'w').write(json.dumps(extra_files_config))

36

In [4]:
files = [
    "all",
    "label",
    "alias",
    "description",
    "item",
    "qualifiers",
    "p31",
    "p279star",
    "derived_P31x"
]
ck = ConfigureKGTK(files, input_files_url=input_files_url)
ck.configure_kgtk(input_graph_path=input_path,
                  output_path=output_path,
                  project_name=project_name,
                  json_config_file = '/root/extra_files.json')

User home: /root
Current dir: /content
KGTK dir: /
Use-cases dir: //use-cases
--2021-11-22 23:12:41--  https://github.com/usc-isi-i2/kgtk-tutorial-files/raw/main/datasets/arnold-profiled/all.tsv.gz
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/usc-isi-i2/kgtk-notebooks/raw/main/datasets/arnold-profiled/all.tsv.gz [following]
--2021-11-22 23:12:41--  https://github.com/usc-isi-i2/kgtk-notebooks/raw/main/datasets/arnold-profiled/all.tsv.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/datasets/arnold-profiled/all.tsv.gz [following]
--2021-11-22 23:12:41--  https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/datasets/arnold-profiled/all.tsv.gz
Resolving raw.githubusercontent.com (raw.gi

In [5]:
ck.print_env_variables()

KGTK_OPTION_DEBUG: false
KGTK_LABEL_FILE: /root/isi-kgtk-tutorial/input/labels.en.tsv.gz
KGTK_GRAPH_CACHE: /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/wikidata.sqlite3.db
USE_CASES_DIR: //use-cases
STORE: /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/wikidata.sqlite3.db
TEMP: /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings
kgtk: kgtk
kypher: kgtk query --graph-cache /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/wikidata.sqlite3.db
EXAMPLES_DIR: //examples
OUT: /tmp/projects/tutorial-graph-embeddings
GRAPH: /root/isi-kgtk-tutorial/input
all: /root/isi-kgtk-tutorial/input/all.tsv.gz
label: /root/isi-kgtk-tutorial/input/labels.en.tsv.gz
alias: /root/isi-kgtk-tutorial/input/aliases.en.tsv.gz
description: /root/isi-kgtk-tutorial/input/descriptions.en.tsv.gz
item: /root/isi-kgtk-tutorial/input/claims.wikibase-item.tsv.gz
qualifiers: /root/isi-kgtk-tutorial/input/qualifiers.tsv.gz
p31: /root/is

In [6]:
ck.load_files_into_cache()

kgtk query --graph-cache /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/wikidata.sqlite3.db -i "/root/isi-kgtk-tutorial/input/all.tsv.gz" --as all  -i "/root/isi-kgtk-tutorial/input/labels.en.tsv.gz" --as label  -i "/root/isi-kgtk-tutorial/input/aliases.en.tsv.gz" --as alias  -i "/root/isi-kgtk-tutorial/input/descriptions.en.tsv.gz" --as description  -i "/root/isi-kgtk-tutorial/input/claims.wikibase-item.tsv.gz" --as item  -i "/root/isi-kgtk-tutorial/input/qualifiers.tsv.gz" --as qualifiers  -i "/root/isi-kgtk-tutorial/input/derived.P31.tsv.gz" --as p31  -i "/root/isi-kgtk-tutorial/input/derived.P279star.tsv.gz" --as p279star  -i "/root/isi-kgtk-tutorial/input/derived.P31x.tsv" --as derived_P31x  --limit 3
node1	label	node2	id	node2;wikidatatype
P10	alias	'gif'@en	P10-alias-en-282226-0	
P10	alias	'animation'@en	P10-alias-en-2f86d8-0	
P10	alias	'media'@en	P10-alias-en-c1427e-0	


In [7]:
# dimension of the output embeddings vector
vector_dimension = 30 

# output path for embeddings file in w2v format
vector_output_w2v_path = f"{os.environ['OUT']}/arnold.embeddings.augmented.{vector_dimension}.w2v.tsv"

os.environ['VECTOR_DIMENSION'] = str(vector_dimension)

## Compute RESCAL Graph Embeddings

In this notebook we will compute graph embeddings using `kgtk graph-embeddings` command for the `arnold` subgraph and demonstrate a few applications.

First step is to augment the `claims.wikibase-item.tsv.gz` file with `derived.P31x.tsv` file which contains occupations for humans as `instance of (P31)`

- `claims.wikibase-item.tsv.gz`: KGTK claims file non literal edges only
- `derived.P31x.tsv`: file with additional P31x links, adding occupation as `instance of` (computed)

In [8]:
!kgtk cat -i $item \
-i $GRAPH/derived.P31x.tsv \
-o $GRAPH/claims.wikibase-item.augmented.tsv.gz

### Run `kgtk graph-embeddings`

The `kgtk graph-embeddings` command takes as input a KGTK edge file and computes graph embeddings of user specified type, producing vectors of user specified dimensions.

The following parameters are used in this instance:

- `-op RESCAL`: compute RESCAL graph embeddings
- `--dimension 30`: desired dimension of the vectors
- `-ot w2v`: output format - w2v
- `--retain_temporary_data True`: retain the byproduct files, which we will use in subsequent steps
- `-T <folder path>`: temporary folder where the temporary files will be stored
- `-i <file>`: input file
- `-o <file>`: output file
- `--log <file>`: log file

**NOTE**: This cell will take ~15 minutes to run on the default Google Colab VM.

In [9]:
kgtk(f""" graph-embeddings
            -op RESCAL \
            --dimension $VECTOR_DIMENSION \
            -ot w2v \
            -i $GRAPH/claims.wikibase-item.augmented.tsv.gz \
            -o {vector_output_w2v_path} \
            --log $TEMP/ge.log.txt
    """)

In Processing, Please go to /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/ge.log.txt to check details
Processed Finished.



#### Take a peek at the embeddings file.

In [10]:
kgtk(f"""head -i {vector_output_w2v_path}""")

In input header '66014 30': 
Invalid Quantity



,66014 30
0,Q102813 0.018146900 0.036498435 0.009308694 0....
1,Q530602 0.006011726 0.022043444 0.028451664 -0...
2,P2142 -0.026932813 -0.028992705 -0.016821105 0...
3,P2868 -0.030970417 -0.015319941 0.004308454 0....
4,Q105084753 0.021836128 -0.040427517 0.02192076...
5,Q4454536 -0.019108273 -0.050377581 -0.03475776...
6,Q1727 -0.040094465 -0.020891000 -0.067256108 0...
7,Q1021106 -0.006659701 -0.068740696 -0.00048465...
8,Q524273 -0.020750102 0.023095433 -0.018623037 ...
9,Q29824585 -0.036862977 -0.002013047 -0.0282932...


O header do w2v precisa conter o número de nós e o numero de dimensões. O vetor é separado por espaços. 

### Load the vectors into `gensim`

To find similar vectors based on cosine similarity

In [11]:
ge_vectors = KeyedVectors.load_word2vec_format(f"{vector_output_w2v_path}", binary=False)

Define a function to compute the `topn` similar vectors, and get the labels and descriptions of the matching Qnodes.

Dado um vetor de entrada contendo o ID e os embeddings a função abaixo do pacote gensim retorna para um ID (positive) os topn IDs mais semelhantes. 

vectors.most_similar(positive=positive, topn=topn)

In [12]:
def kgtk_most_similar(
    vectors,
    positive,
    relation_label="similarity_score",
    add_label_description=True,
    output_path=None,
    topn=25,
):
    """
    find topn similar Qnodes, add label and decription for the Qnodes
    
    :param vectors: vector space loaded into gensim KeyedVectors model
    :param positive: vector(s) or Qnode(s) to find similar entities for
    :param relation_label: name of the property to be used for the output file
    :param add_label_description: boolean parameter to add label and description for matched entities
    :param output_path: path to store the output file
    :param topn: desirednumber of similar entities
    """
    result = []
    if add_label_description:
        fp = tempfile.NamedTemporaryFile(
            mode="w", suffix=".tsv", delete=False, encoding="utf-8"
        )
        fp.write("node1\tlabel\tnode2\n")
        for (qnode, similarity) in vectors.most_similar(positive=positive, topn=topn):
            fp.write("{}\t{}\t{}\n".format(qnode, relation_label, similarity))
        filename = fp.name
        fp.close()

        os.environ["_temp_file"] = filename

        result = !$kypher -i label -i description -i "$_temp_file" --as sim \
--match 'sim: (n1)-[]->(similarity), label: (n1)-[]->(lab), description: (n1)-[]->(des)' \
--return 'distinct n1 as node1, similarity as node2, "similarity" as label, lab as `node1;label`, des as `node1;description`' \
--order-by 'cast(similarity, float) desc' 
        
        os.remove(filename)
        
    else:
        result.append("node1\tlabel\tnode2\n")
        for (qnode, similarity) in vectors.most_similar(positive=positive, topn=topn):
            result.append("{}\t{}\t{}\n".format(qnode, relation_label, similarity))

    if output_path:
        handle = open(output_path, "w")
        for line in result:
            handle.write(line)
            handle.write("\n")
        handle.close()
    else:
        columns = result[0].split("\t")
        data = []
        for line in result[1:]:
            data.append(line.split("\t"))
        return pd.DataFrame(data, columns=columns)

The following function takes as input a `Qnode` and outputs their respective embeddings vector which should be similar to the value of the relation.



In [15]:
def get_embed(head, relation=None):

    if relation is None:
        return ge_vectors[head]
    return 

#### Get the vector for `Q37079` (Tom Cruise) then find most similar entities

a função get_embed recebe o ID do Qnode e retorna o vetor de embeddings desse. Depois esse vetor é usado como o referencial de entrada para a kgtk_most_similar (parametro positive) para obter os 10 (topn) mais similares.

In [17]:
_vector = get_embed('Q37079')
kgtk_most_similar(ge_vectors, positive=[_vector], topn=10)

,node1,node2,label,node1;label,node1;description
0,Q37079,1.0,similarity,'Tom Cruise'@en,'American actor and producer'@en
1,Q42930,0.8838955760002136,similarity,'Dustin Hoffman'@en,'American actor'@en
2,Q734944,0.8556530475616455,similarity,'Clifford Odets'@en,"'American playwright, screenwriter, director a..."
3,Q174843,0.8500082492828369,similarity,'Jeff Bridges'@en,"'American actor, singer-songwriter and produce..."
4,Q103876,0.8471978902816772,similarity,'Peter O\'Toole'@en,'British stage and film actor of Irish descent...
5,Q219546,0.84708172082901,similarity,'Albert Finney'@en,'English actor'@en
6,Q165296,0.8426931500434875,similarity,'Wendi McLendon-Covey'@en,'American actress'@en
7,Q19561563,0.8414174914360046,similarity,'Tim Miller'@en,'American film director'@en
8,Q76478,0.8397270441055298,similarity,'Kirsten Dunst'@en,'American actress'@en
9,Q65932,0.8388870358467102,similarity,'Anthony Hopkins'@en,'Welsh actor'@en


#### Get the vector for `Q170564` (Terminator 2: Judgement Day) , then find most similar entities

In [18]:
_vector = get_embed('Q170564')
kgtk_most_similar(ge_vectors, positive=[_vector], topn=10)

,node1,node2,label,node1;label,node1;description
0,Q170564,0.9999999403953552,similarity,'Terminator 2: Judgment Day'@en,'1991 American science fiction action film dir...
1,Q1125282,0.9086160063743591,similarity,'Man on the Moon'@en,'1999 film by Miloš Forman'@en
2,Q787131,0.8973479270935059,similarity,'BAFTA Award for Best Direction'@en,'award'@en
3,Q913324,0.8920871615409851,similarity,'Legends of the Fall'@en,'1994 American drama film directed by Edward Z...
4,Q3023357,0.8825584053993225,similarity,'12 Years a Slave'@en,'2013 British-American historical drama film d...
5,Q1215857,0.8821021914482117,similarity,"'The Russians Are Coming, the Russians Are Com...",'1966 film by Norman Jewison'@en
6,Q42198,0.8806610703468323,similarity,'Heat'@en,'1995 film directed by Michael Mann'@en
7,Q15963780,0.8798075914382935,similarity,'Carol'@en,'2015 British-American romantic drama film dir...
8,Q1587353,0.8785383701324463,similarity,'The Naked City'@en,'1948 film by Jules Dassin'@en
9,Q1198780,0.8753301501274109,similarity,'The Great Caruso'@en,'1951 film by Richard Thorpe'@en


#### Get the vector for `Q104123` (Pulp Fiction), then find most similar entities

In [19]:
_vector = get_embed('Q104123')
kgtk_most_similar(ge_vectors, positive=[_vector], topn=10)

,node1,node2,label,node1;label,node1;description
0,Q104123,1.0,similarity,'Pulp Fiction'@en,'1994 American crime film directed by Quentin ...
1,Q211372,0.9616209864616394,similarity,'On the Waterfront'@en,'1954 film by Elia Kazan'@en
2,Q1218959,0.9516146779060364,similarity,'The Last Picture Show'@en,'1971 film by Peter Bogdanovich'@en
3,Q826494,0.9396955370903015,similarity,'Shine'@en,'1996 Australian film directed by Scott Hicks'@en
4,Q736969,0.9391366839408875,similarity,'Guess Who\'s Coming to Dinner'@en,'1967 film by Stanley Kramer'@en
5,Q308929,0.9390137791633606,similarity,'Training Day'@en,'2001 film directed by Antoine Fuqua'@en
6,Q181069,0.9389439821243286,similarity,'Star Wars: Episode II – Attack of the Clones'@en,'2002 American epic space opera film directed ...
7,Q474093,0.9362003207206726,similarity,'The Girl with the Dragon Tattoo'@en,'2011 film by David Fincher'@en
8,Q817159,0.9349483847618103,similarity,'All the King\'s Men'@en,'1949 American film based on Robert Penn Warre...
9,Q936576,0.9330049753189087,similarity,'The Nutty Professor'@en,'1996 film by Tom Shadyac'@en


#### Get the vector for `Q2685` (Arnold Schwarzenegger), then find most similar entities

a função get_embed recebe o ID do nó e retorna o vetor de embeddings desse ID. Depois esse vetor é usado como o referencial de entrada para a kgtk_most_similar (parametro positive) para obter os 10 (topn) mais similares.

In [20]:
_vector = get_embed('Q2685')
kgtk_most_similar(ge_vectors, positive=[_vector], topn=10)

,node1,node2,label,node1;label,node1;description
0,Q2685,1.0,similarity,'Arnold Schwarzenegger'@en,"'Austrian-American actor, businessman, bodybui..."
1,Q5694035,0.7956441640853882,similarity,'Heather Parry'@en,'American film and television producer'@en
2,Q712450,0.7879326939582825,similarity,'Ralph Waite'@en,'American actor'@en
3,Q352010,0.7831586599349976,similarity,'David S. Goyer'@en,"'American screenwriter, film director, novelis..."
4,Q34460,0.7713244557380676,similarity,'Gwyneth Paltrow'@en,"'American actress, entrepreneur, writer and si..."
5,Q156552,0.7622668743133545,similarity,'Lynn Redgrave'@en,'British-American actress'@en
6,Q317567,0.7613799571990967,similarity,'Barry Sonnenfeld'@en,'American film director and cinematographer'@en
7,Q125106,0.7601830363273621,similarity,'Salma Hayek'@en,"'Lebanese, Mexican, and American actress and p..."
8,Q2680,0.7591208219528198,similarity,'Bruce Willis'@en,'American actor and film producer'@en
9,Q1275768,0.7565523982048035,similarity,'Fred Rogers'@en,'American television personality'@en


#### Get the vector for `Q103148` (Lahn River), then find most similar entities

In [21]:
_vector = get_embed('Q103148')
kgtk_most_similar(ge_vectors, positive=[_vector], topn=10)

,node1,node2,label,node1;label,node1;description
0,Q103148,1.0,similarity,'Lahn'@en,'right tributary of the Rhine River in Germany...
1,Q820228,0.9099758267402649,similarity,'Brenz'@en,'river in Germany'@en
2,Q328429,0.9076910614967346,similarity,'Leibi'@en,'river in Germany'@en
3,Q168696,0.8981105089187622,similarity,'Nahe'@en,'tributary of Rhine river'@en
4,Q208302,0.8980113863945007,similarity,'Siret River'@en,'river in Romania and Ukraine'@en
5,Q985,0.8965878486633301,similarity,'Baden-Württemberg'@en,'federal state in southwestern Germany'@en
6,Q217805,0.8958309888839722,similarity,'Iskar'@en,'river in Bulgaria'@en
7,Q705057,0.8944600820541382,similarity,'Sauer'@en,'river of France and Germany'@en
8,Q216171,0.8942854404449463,similarity,'Waal'@en,'river in the Netherlands'@en
9,Q702507,0.8929464221000671,similarity,'Naab'@en,'river in Germany'@en
